<a href="https://www.kaggle.com/omerparlak/cltv-analysis-with-bg-nbd-gg-model?scriptVersionId=88187847" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Business Problem**

### An e-commerce website wants a forward projection for customer actions according to the CLTV of its customers.

### With the dataset in your hands, is it possible to identify the customers who can generate the most revenue within 1, 6 and 12 month time periods?

### This study was conducted for UK customers data from 2010-2011.


In [1]:
pip install Lifetimes

     |████████████████████████████████| 584 kB 590 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47989 sha256=4bc3cdeffaf517fd5f0e3ee7f2a2ff62ac685bc3caa121dd6552bebf8eeb9710
  Stored in directory: /root/.cache/pip/wheels/ef/32/31/0e87227cd0ca1d99ad51fbe4b54c6fa02afccf7e483d045e04
Successfully built autograd
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

In [3]:
pip install openpyxl

     |████████████████████████████████| 242 kB 592 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
df_ = pd.read_excel("../input/online-retail-ll/online_retail_II.xlsx",sheet_name="Year 2010-2011")

In [5]:
df = df_.copy()

In [6]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################", end="\n\n")
    print(dataframe.shape, end="\n\n")
    print("##################### Types #####################", end="\n\n")
    print(dataframe.dtypes, end="\n\n")
    print("##################### Head #####################", end="\n\n")
    print(dataframe.head(head), end="\n\n")
    print("##################### Tail #####################", end="\n\n")
    print(dataframe.tail(head), end="\n\n")
    print("##################### NA #####################", end="\n\n")
    print(dataframe.isnull().sum(), end="\n\n")
    print("##################### Quantiles #####################", end="\n\n")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T, end="\n\n")


def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    

def data_prep(data, country):
    data = data.rename(columns={"Customer ID": "CustomerID"})
    today_date = dt.datetime(2011, 12, 11)
    data = data.dropna(axis=0)
    data = data[~data["Invoice"].str.contains("C", na=False)]
    data = data[~data["Description"].str.contains("POSTAGE", na=False)]
    data = data[(data['Quantity'] > 0)]
    data = data[(data['Price'] > 0)]
    replace_with_thresholds(data, "Quantity")
    replace_with_thresholds(data, "Price")
    data["TotalPrice"] = data["Price"] * data["Quantity"]
    data["CustomerID"] = data["CustomerID"].astype("int")
    data = data.groupby('CustomerID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                                     lambda date: (today_date - date.min()).days],
                                                     'Invoice': lambda num: num.nunique(),
                                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
    data.columns = data.columns.droplevel(0)
    data.columns = ['recency', 'T', 'frequency', 'monetary']
    data["recency"] = data["recency"] / 7
    data["T"] = data["T"] / 7
    data["monetary"] = data["monetary"] / data["frequency"]
    data = data[(data['frequency'] > 1)]
    data = data[data["monetary"] > 0]
    return data


def cltv_calculator(month=1):
    cltv_x = ggf.customer_lifetime_value(bgf,
                                     cltv_df['frequency'],
                                     cltv_df['recency'],
                                     cltv_df['T'],
                                     cltv_df['monetary'],
                                     time=month,    
                                     freq="W",  # Frequency information of T
                                     discount_rate=0.01)
    cltv_x = cltv_x.reset_index()
    cltv_x = cltv_x.rename(columns={"clv": "clv_" + str(month)})
    return cltv_x

In [7]:
check_df(df)

##################### Shape #####################

(541910, 8)

##################### Types #####################

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

##################### Head #####################

  Invoice StockCode                          Description  Quantity  \
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                  WHITE METAL LANTERN         6   
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39      17850

In [8]:
cltv_df = data_prep(df, "United Kingdom")
cltv_df

,recency,T,frequency,monetary
CustomerID,,,,
12347,52.142857,52.571429,7,615.714286
12348,40.285714,51.285714,4,359.310000
12352,37.142857,42.428571,8,185.052500
12356,43.142857,46.571429,3,829.143333
12358,21.285714,21.571429,2,464.030000
...,...,...,...,...
18272,34.857143,35.285714,6,513.096667
18273,36.428571,36.857143,3,68.000000
18282,16.857143,18.142857,2,89.025000


# **BG-NBD Model**

In [9]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

bgf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
r,2.184633,0.075739,2.036184,2.333081
alpha,11.477788,0.463120,10.570073,12.385502
a,0.116732,0.012600,0.092035,0.141428
b,2.480305,0.229812,2.029874,2.930736


# **GAMMA-GAMMA Model**

In [10]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
ggf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
p,3.798630,0.097246,3.608027,3.989232
q,0.342553,0.007275,0.328294,0.356813
v,3.732703,0.097829,3.540957,3.924449


# **CLTV Prediction with BG-NBD Model and GG Model**

In [11]:
cltv_1 = cltv_calculator(month=1)
cltv_1

,CustomerID,clv_1
0,12347,382.604988
1,12348,148.517723
2,12352,148.159886
3,12356,325.381989
4,12358,266.274893
...,...,...
2832,18272,389.507470
2833,18273,32.823679
2834,18282,57.595801
2835,18283,171.405411


In [12]:
cltv_6 = cltv_calculator(month=6)
cltv_6

,CustomerID,clv_6
0,12347,2199.946758
1,12348,853.334602
2,12352,849.579292
3,12356,1866.660438
4,12358,1508.988483
...,...,...
2832,18272,2227.177678
2833,18273,187.675696
2834,18282,325.478800
2835,18283,984.944300


In [13]:
cltv_12 = cltv_calculator(month=12)
cltv_12

,CustomerID,clv_12
0,12347,4199.318639
1,12348,1627.891226
2,12352,1618.235747
3,12356,3556.676157
4,12358,2850.654481
...,...,...
2832,18272,4233.215174
2833,18273,356.697288
2834,18282,613.793542
2835,18283,1879.157800


## **Merging Tables**

In [14]:
cltv_final = cltv_df.merge(cltv_1,on="CustomerID", how="left")
cltv_final = cltv_final.merge(cltv_6,on="CustomerID", how="left")
cltv_final = cltv_final.merge(cltv_12,on="CustomerID", how="left")

In [15]:
cltv_final.head()

,CustomerID,recency,T,frequency,monetary,clv_1,clv_6,clv_12
0,12347,52.142857,52.571429,7,615.714286,382.604988,2199.946758,4199.318639
1,12348,40.285714,51.285714,4,359.310000,148.517723,853.334602,1627.891226
2,12352,37.142857,42.428571,8,185.052500,148.159886,849.579292,1618.235747
3,12356,43.142857,46.571429,3,829.143333,325.381989,1866.660438,3556.676157
4,12358,21.285714,21.571429,2,464.030000,266.274893,1508.988483,2850.654481


# **Segmentation of  12-Month CLTV Prediction**

In [16]:
scaler = MinMaxScaler((1, 100))
scaler.fit(cltv_final[["clv_12"]])
cltv_final["scaled_clv_12"] = scaler.transform(cltv_final[["clv_12"]])
cltv_final["Segment_for_12"] = pd.qcut(cltv_final["scaled_clv_12"], 4, labels=["D", "C", "B", "A"])

In [17]:
cltv_final.head()

,CustomerID,recency,T,frequency,monetary,clv_1,clv_6,clv_12,scaled_clv_12,Segment_for_12
0,12347,52.142857,52.571429,7,615.714286,382.604988,2199.946758,4199.318639,3.011680,A
1,12348,40.285714,51.285714,4,359.310000,148.517723,853.334602,1627.891226,1.779840,C
2,12352,37.142857,42.428571,8,185.052500,148.159886,849.579292,1618.235747,1.775214,C
3,12356,43.142857,46.571429,3,829.143333,325.381989,1866.660438,3556.676157,2.703823,A
4,12358,21.285714,21.571429,2,464.030000,266.274893,1508.988483,2850.654481,2.365604,B


In [18]:
# Best customers
cltv_final.sort_values(by="scaled_clv_12", ascending=False).head(10)

,CustomerID,recency,T,frequency,monetary,clv_1,clv_6,clv_12,scaled_clv_12,Segment_for_12
1115,14646,50.428571,50.714286,72,3680.965625,18816.870700,108233.386920,206659.385859,100.000000,A
2753,18102,52.285714,52.571429,60,3857.466417,16055.267853,92387.521433,176463.304361,85.534593,A
836,14096,13.857143,14.571429,17,3132.543529,9827.170895,55486.243834,104591.265395,51.104355,A
35,12415,44.714286,48.285714,20,6008.664750,9437.212142,54233.645046,103478.683539,50.571374,A
1250,14911,53.142857,53.428571,201,689.225572,9241.494174,53194.226695,101626.927053,49.684291,A
2450,17450,51.285714,52.571429,46,2863.274891,9205.500841,52969.494775,101170.233300,49.465513,A
867,14156,51.571429,53.142857,55,2101.124182,7926.383475,45616.580285,87137.681303,42.743231,A
2479,17511,52.857143,53.428571,31,2933.943065,6433.443584,37023.122409,70719.848640,34.878282,A
2068,16684,50.428571,51.285714,28,2209.969107,4555.216607,26200.467884,50025.491970,24.964669,A
643,13694,52.714286,53.428571,50,1275.700500,4393.146565,25283.977938,48299.943992,24.138046,A


In [19]:
cltv_final.drop("CustomerID", axis=1).groupby("Segment_for_12").agg({"count", "mean", "sum"})

recency                              T                \
                 count           sum       mean count           sum   
Segment_for_12                                                        
D                  710  15719.285714  22.139839   710  28794.142857   
C                  709  21536.142857  30.375378   709  26770.571429   
B                  709  20948.857143  29.547048   709  24696.000000   
A                  709  22618.428571  31.901874   709  24814.714286   

                          frequency                  monetary  ...  \
                     mean     count   sum       mean    count  ...   
Segment_for_12                                                 ...   
D               40.555131       710  2179   3.069014      710  ...   
C               37.758211       709  2846   4.014104      709  ...   
B               34.832158       709  3866   5.452750      709  ...   
A               34.999597       709  8088  11.407616      709  ...   

                     clv_1 clv_6                            clv_12  \
                      mean count           sum         mean  count   
Segment_for_12                                                       
D                48.558912   710  1.970859e+05   277.585752    710   
C               128.362512   709  5.197321e+05   733.049564    709   
B               232.154506   709  9.379869e+05  1322.971718    709   
A               750.206609   709  3.036203e+06  4282.373065    709   

                                          scaled_clv_12                         
                         sum         mean         count          sum      mean  
Segment_for_12                                                                  
D               3.746197e+05   527.633335           710   889.461228  1.252762  
C               9.873012e+05  1392.526408           709  1181.965797  1.667089  
B               1.779596e+06  2510.008893           709  1561.514070  2.202418  
A               5.768405e+06  8135.973315           709  3472.349463  4.897531  

[4 rows x 24 columns]

In [20]:
# We can get some actions based on segment analysis like loyality programs, campaigns, gift coupons, reminder messages etc.